### Update 28.07.2021

The files structure is quite messed up here. I'm going to label each of these to make writing up easier. This file is:

- PyTorch tutorial / source code solution with extensions
- [source - code](https://pytorch.org/vision/stable/_modules/torchvision/datasets/mnist.html#MNIST)
- [source - tutorial](https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html)


Based on PyTorch tutorial [link](https://pytorch.org/tutorials/beginner/basics/intro.html)

Essentially this solves the problem, it creates a model for analysing handwritten integers. The problem is, I don't fully understand the mechanism it uses.

Source code for PyTorch MNIST can be found here: [link](https://pytorch.org/vision/stable/_modules/torchvision/datasets/mnist.html#MNIST)

In [1]:
# Using PyTorch to see if I can complete CH4 project

import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt

In [2]:
# This command does not appear to work, when I attempt to transform
# data sets into tensors

training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [3]:
training_data

Dataset MNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [4]:
test_data

Dataset MNIST
    Number of datapoints: 10000
    Root location: data
    Split: Test
    StandardTransform
Transform: ToTensor()

In [5]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print("Shape of X [N, C, H, W]: ", X.shape)
    print("Shape of y: ", y.shape, y.dtype)
    break

Shape of X [N, C, H, W]:  torch.Size([64, 1, 28, 28])
Shape of y:  torch.Size([64]) torch.int64


In [6]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


In [7]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [8]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [9]:
def test(dataloader, model):
    size = len(dataloader.dataset)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


In [10]:
epochs = 20
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model)
print("Done!")

Epoch 1
-------------------------------
loss: 2.302561  [    0/60000]
loss: 2.296736  [ 6400/60000]
loss: 2.294334  [12800/60000]
loss: 2.286954  [19200/60000]
loss: 2.295547  [25600/60000]
loss: 2.288713  [32000/60000]
loss: 2.283125  [38400/60000]
loss: 2.286916  [44800/60000]
loss: 2.278717  [51200/60000]
loss: 2.263468  [57600/60000]
Test Error: 
 Accuracy: 43.5%, Avg loss: 0.035644 

Epoch 2
-------------------------------
loss: 2.272403  [    0/60000]
loss: 2.263773  [ 6400/60000]
loss: 2.260313  [12800/60000]
loss: 2.243842  [19200/60000]
loss: 2.263381  [25600/60000]
loss: 2.255928  [32000/60000]
loss: 2.236527  [38400/60000]
loss: 2.253817  [44800/60000]
loss: 2.234027  [51200/60000]
loss: 2.210589  [57600/60000]
Test Error: 
 Accuracy: 56.3%, Avg loss: 0.034920 

Epoch 3
-------------------------------
loss: 2.227485  [    0/60000]
loss: 2.215161  [ 6400/60000]
loss: 2.213765  [12800/60000]
loss: 2.180887  [19200/60000]
loss: 2.218458  [25600/60000]
loss: 2.208467  [32000/600

In [11]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


Tutorial for using PyTorch with your own data can be found here: [link](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html)